# Bottom Up Tree Search Synthesizer and SMT Based Verification Oracle

The Bottom Up Tree Search Synthesizer accepts a CFG written in sympy expressions and synthesizes sympy expressions, while the SMT Based Verification Oracle operates on z3 expressions.

## Initializing `verification_oracle_instance`

In [1]:
from z3 import *

from verification_oracle import verification_oracle


z3_x = Int('x')
z3_y = Int('y')

z3_input_variable_list = [z3_x, z3_y]
z3_function_declaration = max2 = Function('max2', IntSort(), IntSort(), IntSort())
z3_constraint = And(max2(z3_x, z3_y) >= z3_x, max2(z3_x, z3_y) >= z3_y, Or(max2(z3_y, z3_x) == z3_x, max2(z3_y, z3_x) == z3_y))

verification_oracle_instance = verification_oracle(z3_input_variable_list, z3_function_declaration, z3_constraint)
next(verification_oracle_instance)

## Initializing `bottom_up_tree_search_instance`

In [2]:
from sympy import *

from bottom_up_tree_search import bottom_up_tree_search


S = symbols('S')
B = symbols('B')
x = symbols('x')
y = symbols('y')

non_terminals = { S, B }
terminals = { x, y }
production_rules = {
  S: {
    x,
    y,
    Add(S, S, evaluate=False),
    Add(S, -S, evaluate=False),
    Piecewise((S, B), (S, True), evaluate=False)
  },
  B: {
    Le(S, S, evaluate=False),
    Eq(S, S, evaluate=False),
    Ge(S, S, evaluate=False)
  }
}
start_symbol = S

function_declaration = max2 = Function('max2')
constraint = And(GreaterThan(max2(x, y), x), GreaterThan(max2(x, y), y), Or(Equality(max2(y, x), x), Equality(max2(y, x), y)))

bottom_up_tree_search_instance = bottom_up_tree_search(
    non_terminals,
    terminals,
    production_rules,
    start_symbol,
    function_declaration,
    constraint
)

## Conversion between sympy expressions and z3 expressions

In [3]:
from sympy_expr_to_z3_expr_ref import sympy_expr_to_z3_expr_ref
from z3_expr_ref_to_sympy_expr import z3_expr_ref_to_sympy_expr

sympy_symbols_to_z3_expr_ref = {
    x: z3_x,
    y: z3_y
}

z3_expr_ref_to_sympy_symbols = {
    z3_x: x,
    z3_y: y
}

## Program Synthesis and Verification

In [4]:
from frozendict import frozendict

In [5]:
sympy_expr = next(bottom_up_tree_search_instance)

non_terminal S non_leaf_production_rule Piecewise((S, B), (S, True))
non_terminal S non_leaf_production_rule S + S
non_terminal S non_leaf_production_rule -S + S
non_terminal B non_leaf_production_rule Eq(S, S)
non_terminal B non_leaf_production_rule S <= S
non_terminal B non_leaf_production_rule S >= S
non_terminals_to_expression_sizes_to_expressions {S: {1: {x: {}, y: {}}}}


In [6]:
print(f'sympy_expr={sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = bottom_up_tree_search_instance.send(counterexample_input)
    
print(f'z3_counterexample_input={z3_counterexample_input}')

sympy_expr=x
z3_counterexample_input={x: -1, y: 0}


In [7]:
print(f'sympy_expr={sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = bottom_up_tree_search_instance.send(counterexample_input)
    
print(f'z3_counterexample_input={z3_counterexample_input}')

non_terminal S non_leaf_production_rule Piecewise((S, B), (S, True))
non_terminal S non_leaf_production_rule S + S
	 sum_of_k_integers (1, 1)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, y)
non_terminal S non_leaf_production_rule -S + S
	 sum_of_k_integers (1, 1)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, y)
non_terminal B non_leaf_production_rule Eq(S, S)
	 sum_of_k_integers (1, 1)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, y)
		 expressions_for_non_termi

sympy_expr=y


		 expressions_for_non_terminals_in_non_leaf_production_rule (y, y)
non_terminal B non_leaf_production_rule S <= S
	 sum_of_k_integers (1, 1)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, y)
non_terminal B non_leaf_production_rule S >= S
	 sum_of_k_integers (1, 1)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, y)


skipped 2*x
skipped x + y
skipped 2*y
z3_counterexample_input={x: 0, y: -1}


		 expressions_for_non_terminals_in_non_leaf_production_rule (y, x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, y)
non_terminals_to_expression_sizes_to_expressions {S: {1: {x: {}, y: {}}, 2: {2*x: {}, x + y: {}, 2*y: {}, 0: {}, x - y: {}, -x + y: {}}}, B: {2: {True: {}, Eq(x, y): {}, x <= y: {}, x >= y: {}}}}


In [8]:
print(f'sympy_expr={sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = bottom_up_tree_search_instance.send(counterexample_input)
    
print(f'z3_counterexample_input={z3_counterexample_input}')

non_terminal S non_leaf_production_rule Piecewise((S, B), (S, True))
	 sum_of_k_integers (1, 1, 1)
non_terminal S non_leaf_production_rule S + S
	 sum_of_k_integers (1, 2)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, 2*x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, x + y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, 2*y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, 0)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, x - y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, -x + y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, 2*x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, x + y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, 2*y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, 0)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, x - y)
		 expressions_for_non_terminals_in_non_leaf_produc

sympy_expr=0
skipped x - y
skipped -x + y


		 expressions_for_non_terminals_in_non_leaf_production_rule (x + y, y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (2*y, x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (2*y, y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (0, x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (0, y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x - y, x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x - y, y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (-x + y, x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (-x + y, y)
non_terminal B non_leaf_production_rule S <= S
	 sum_of_k_integers (1, 2)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, 2*x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, x + y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, 2*y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (x, 0)
		 expre

skipped 3*x
skipped 2*x + y
skipped x + 2*y
skipped 2*x - y
skipped 3*y
skipped -x + 2*y
skipped -x
skipped -y
skipped x - 2*y
skipped -2*x + y


		 expressions_for_non_terminals_in_non_leaf_production_rule (y, x + 2*y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, 2*x - y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, 3*y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, -x + 2*y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, -x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, -y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, x - 2*y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (y, -2*x + y)
	 sum_of_k_integers (2, 2)
		 expressions_for_non_terminals_in_non_leaf_production_rule (2*x, 2*x)
		 expressions_for_non_terminals_in_non_leaf_production_rule (2*x, x + y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (2*x, 2*y)
		 expressions_for_non_terminals_in_non_leaf_production_rule (2*x, 0)
		 expressions_for_non_terminals_in_non_leaf_production_rule (2*x, x - y)
		 expressions_for_non_terminal

skipped Piecewise((x, x <= y), (y, True))
z3_counterexample_input={x: -1, y: -1}


non_terminals_to_expression_sizes_to_expressions {S: {1: {x: {}, y: {}}, 2: {2*x: {}, x + y: {}, 2*y: {}, 0: {}, x - y: {}, -x + y: {}}, 3: {3*x: {}, 2*x + y: {}, x + 2*y: {}, 2*x - y: {}, 3*y: {}, -x + 2*y: {}, -x: {}, -y: {}, x - 2*y: {}, -2*x + y: {}}, 4: {Piecewise((x, x <= y), (y, True)): {}, Piecewise((x, x >= y), (y, True)): {}, Piecewise((y, x <= y), (x, True)): {}, Piecewise((y, x >= y), (x, True)): {}, 4*x: {}, 3*x + y: {}, 2*x + 2*y: {}, 3*x - y: {}, x + 3*y: {}, 2*x - 2*y: {}, 4*y: {}, -x + 3*y: {}, -2*x + 2*y: {}, -2*x: {}, -x - y: {}, -2*y: {}, x - 3*y: {}, -3*x + y: {}}}, B: {2: {True: {}, Eq(x, y): {}, x <= y: {}, x >= y: {}}, 1: {}, 3: {Eq(x, 0): {}, Eq(y, 0): {}, Eq(x, 2*y): {}, Eq(y, 2*x): {}, x >= 0: {}, y >= 0: {}, x <= 2*y: {}, x <= 0: {}, y <= 0: {}, y >= 2*x: {}, 2*x >= y: {}, x >= 2*y: {}, 2*x <= y: {}, y <= 2*x: {}}, 4: {Eq(x, -y): {}, Eq(x, 3*y): {}, Eq(y, 3*x): {}, x >= -y: {}, x <= 3*y: {}, x <= -y: {}, y >= 3*x: {}, 3*x >= y: {}, x >= 3*y: {}, y <= 3*x: {}

In [9]:
print(f'sympy_expr={sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = bottom_up_tree_search_instance.send(counterexample_input)
    
print(f'z3_counterexample_input={z3_counterexample_input}')

sympy_expr=Piecewise((x, x >= y), (y, True))
z3_counterexample_input=None
